In [1]:
!pip install hdfs
!pip install prettytable

In [2]:
import pandas as pd
import numpy as np
from pyspark import SparkConf
from pyspark.context import SparkContext
from pyspark.sql import SparkSession
from hdfs import InsecureClient
from pyspark.sql.functions import col, sum, when
from prettytable import PrettyTable
import matplotlib.pyplot as plt
import seaborn as sns

sc = SparkContext.getOrCreate(SparkConf().setMaster("local[*]"))
spark = SparkSession.builder.appName("dataproject").config("spark.hadoop.fs.defaultFS", "hdfs://namenode:9000").getOrCreate()



In [3]:
event_data = spark.read.option("header", "true").option("multiline", "true").option("escape", "\"").csv("hdfs://namenode:9000/ufc_event_data.csv")
fight_data = spark.read.option("header", "true").option("multiline", "true").option("escape", "\"").csv("hdfs://namenode:9000/ufc_fight_data.csv")
fight_stats = spark.read.option("header", "true").option("multiline", "true").option("escape", "\"").csv("hdfs://namenode:9000/ufc_fight_stat_data.csv")
fighter_data = spark.read.option("header", "true").option("multiline", "true").option("escape", "\"").csv("hdfs://namenode:9000/ufc_fighter_data.csv")


<h1>Exploration and Preprocessing</h1>

In [4]:
event_data.printSchema()

root
 |-- event_id: string (nullable = true)
 |-- event_name: string (nullable = true)
 |-- event_date: string (nullable = true)
 |-- event_city: string (nullable = true)
 |-- event_state: string (nullable = true)
 |-- event_country: string (nullable = true)
 |-- event_url: string (nullable = true)



In [5]:
event_data_pdf = event_data.toPandas()
null_counts = event_data_pdf.isnull().sum()
print(null_counts)
event_data_pdf.describe()

event_id          0
event_name        0
event_date        0
event_city        0
event_state      49
event_country     0
event_url         0
dtype: int64


,event_id,event_name,event_date,event_city,event_state,event_country,event_url
count,665,665,665,665,616,665,665
unique,665,665,660,162,83,27,665
top,134,UFC 282: Blachowicz vs. Ankalaev,2014-08-23,Las Vegas,Nevada,USA,http://ufcstats.com/event-details/896c322f56b8...
freq,1,1,2,222,222,469,1


In [6]:
event_data = event_data.drop("event_state", "event_country", "event_url")

In [7]:
fight_data.printSchema()

root
 |-- fight_id: string (nullable = true)
 |-- event_id: string (nullable = true)
 |-- referee: string (nullable = true)
 |-- f_1: string (nullable = true)
 |-- f_2: string (nullable = true)
 |-- winner: string (nullable = true)
 |-- num_rounds: string (nullable = true)
 |-- title_fight: string (nullable = true)
 |-- weight_class: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- result: string (nullable = true)
 |-- result_details: string (nullable = true)
 |-- finish_round: string (nullable = true)
 |-- finish_time: string (nullable = true)
 |-- fight_url: string (nullable = true)



In [8]:
fight_data_pdf = fight_data.toPandas()

null_counts = fight_data_pdf.isnull().sum()

print(null_counts)
fight_data_pdf.describe()

fight_id           0
event_id           0
referee           32
f_1               19
f_2               13
winner            15
num_rounds         0
title_fight        0
weight_class      13
gender             0
result             0
result_details    17
finish_round       0
finish_time        0
fight_url          0
dtype: int64


,fight_id,event_id,referee,f_1,f_2,winner,num_rounds,title_fight,weight_class,gender,result,result_details,finish_round,finish_time,fight_url
count,7218,7218,7186,7199,7205,7203,7218,7218,7205,7218,7218,7201,7218,7218,7218
unique,7218,664,217,1741,2252,1676,5,2,14,2,5,342,5,327,7218
top,5875,1,Herb Dean,1693.0,542.0,1693.0,3,F,Lightweight,M,Decision,Unanimous,3,5:00,http://ufcstats.com/fight-details/d77a72e3fa8d...
freq,1,15,1065,32,20,25,6382,6811,1247,6521,3315,2574,3620,3346,1


In [9]:
fight_data = fight_data.drop("referee", "fight_url", "result_details")
fight_data.dropna(subset=["f_1", "f_2", "winner"])


DataFrame[fight_id: string, event_id: string, f_1: string, f_2: string, winner: string, num_rounds: string, title_fight: string, weight_class: string, gender: string, result: string, finish_round: string, finish_time: string]

In [10]:
fight_stats.printSchema()

root
 |-- fight_stat_id: string (nullable = true)
 |-- fight_id: string (nullable = true)
 |-- fighter_id: string (nullable = true)
 |-- knockdowns: string (nullable = true)
 |-- total_strikes_att: string (nullable = true)
 |-- total_strikes_succ: string (nullable = true)
 |-- sig_strikes_att: string (nullable = true)
 |-- sig_strikes_succ: string (nullable = true)
 |-- takedown_att: string (nullable = true)
 |-- takedown_succ: string (nullable = true)
 |-- submission_att: string (nullable = true)
 |-- reversals: string (nullable = true)
 |-- ctrl_time: string (nullable = true)
 |-- fight_url: string (nullable = true)



In [11]:
fight_stats_pdf = fight_stats.toPandas()

null_counts = fight_stats_pdf.isnull().sum()

print(null_counts)
fight_stats_pdf.describe()

fight_stat_id          0
fight_id               0
fighter_id            32
knockdowns            42
total_strikes_att     42
total_strikes_succ    42
sig_strikes_att       42
sig_strikes_succ      42
takedown_att          42
takedown_succ         42
submission_att        42
reversals             42
ctrl_time             42
fight_url              0
dtype: int64


,fight_stat_id,fight_id,fighter_id,knockdowns,total_strikes_att,total_strikes_succ,sig_strikes_att,sig_strikes_succ,takedown_att,takedown_succ,submission_att,reversals,ctrl_time,fight_url
count,14436,14436,14404,14394,14394,14394,14394,14394,14394,14394,14394,14394,14394,14436
unique,14436,7218,2412,6,416,259,388,199,32,17,11,7,818,7218
top,5875,5875,1693.0,0.0,10.0,1.0,3.0,1.0,0.0,0.0,0.0,0.0,0:00,http://ufcstats.com/fight-details/d77a72e3fa8d...
freq,1,2,41,11741,136,255,171,366,4978,7807,10811,12809,2218,2


In [12]:
fight_stats.dropna()
fight_stats = fight_stats.drop("fight_url")

In [13]:
fighter_data.printSchema()

root
 |-- fighter_id: string (nullable = true)
 |-- fighter_f_name: string (nullable = true)
 |-- fighter_l_name: string (nullable = true)
 |-- fighter_nickname: string (nullable = true)
 |-- fighter_height_cm: string (nullable = true)
 |-- fighter_weight_lbs: string (nullable = true)
 |-- fighter_reach_cm: string (nullable = true)
 |-- fighter_stance: string (nullable = true)
 |-- fighter_dob: string (nullable = true)
 |-- fighter_w: string (nullable = true)
 |-- fighter_l: string (nullable = true)
 |-- fighter_d: string (nullable = true)
 |-- fighter_nc_dq: string (nullable = true)
 |-- fighter_url: string (nullable = true)



In [14]:
fighter_data_pdf = fighter_data.toPandas()

null_counts = fighter_data_pdf.isnull().sum()

print(null_counts)
fighter_data_pdf.describe()


fighter_id               0
fighter_f_name           0
fighter_l_name          15
fighter_nickname      1857
fighter_height_cm      310
fighter_weight_lbs      87
fighter_reach_cm      1941
fighter_stance         834
fighter_dob            758
fighter_w                0
fighter_l                0
fighter_d                0
fighter_nc_dq         3625
fighter_url              0
dtype: int64


,fighter_id,fighter_f_name,fighter_l_name,fighter_nickname,fighter_height_cm,fighter_weight_lbs,fighter_reach_cm,fighter_stance,fighter_dob,fighter_w,fighter_l,fighter_d,fighter_nc_dq,fighter_url
count,4107,4107,4092,2250,3797,4020,2166,3273,3349,4107,4107,4107,482,4107
unique,4107,1861,3099,1779,26,112,27,5,2838,57,40,10,4,4107
top,3773,Chris,Silva,The Beast,182.88,155.0,182.88,Orthodox,1993-03-16,8,3,0,1.0,http://ufcstats.com/fighter-details/de45aaae23...
freq,1,56,31,13,437,618,205,2513,4,255,516,3431,424,1


In [15]:
# Drop irrelavent columns and replace null values
# Disqualifications are rare and thus often written incorrectly as null values on fighter's records, as such
# these nulled no contest disqulification values should be replaced with zeros.
fighter_data = fighter_data.drop("fighter_height_cm", "fighter_weight_lbs", "fighter_reach_cm", "fighter_stance")
fighter_data = fighter_data.fillna({"fighter_nc_dq": 0})

In [16]:
#Cast relevant columns to int to prevent automatic float casting, allowing proper querying.

fight_stats = fight_stats \
    .withColumn("fight_id", fight_stats["fight_id"].cast("int")) \
    .withColumn("fight_stat_id", fight_stats["fight_stat_id"].cast("int")) \
    .withColumn("fighter_id", fight_stats["fighter_id"].cast("int")) \
    .withColumn("submission_att", fight_stats["submission_att"].cast("int")) \
    .withColumn("knockdowns", fight_stats["knockdowns"].cast("int"))

fight_data = fight_data \
    .withColumn("fight_id", fight_data["fight_id"].cast("int")) \
    .withColumn("winner", fight_data["winner"].cast("int")) 

fighter_data = fighter_data \
    .withColumn("fighter_id", fighter_data["fighter_id"].cast("int"))


<h1>Write cleaned data tables to HDFS</h1>

In [17]:
#Write cleaned data tables to HDFS

event_data.write \
        .mode("overwrite") \
        .parquet('hdfs://namenode:9000/cleaned_event_data.csv')

fight_data.write \
        .mode("overwrite") \
        .parquet('hdfs://namenode:9000/cleaned_fight_data.csv')

fight_stats.write \
        .mode("overwrite") \
        .parquet('hdfs://namenode:9000/cleaned_fight_stats.csv')

fighter_data.write \
        .mode("overwrite") \
        .parquet('hdfs://namenode:9000/cleaned_fighter_data.csv')

In [18]:
event_data.createOrReplaceTempView("event_data")
fight_data.createOrReplaceTempView("fight_data")
fight_stats.createOrReplaceTempView("fight_stats")
fighter_data.createOrReplaceTempView("fighter_data")

<h1>Fight Result Analaysis:</h1>

In [ ]:
finish_rate_by_year = spark.sql("""
    SELECT 
        YEAR(event_date) AS year,
        COUNT(CASE WHEN f.result = 'KO/TKO' OR f.result = 'Submission' THEN 1 END) AS total_finishes,
        COUNT(f.fight_id) AS total_fights,
        (COUNT(CASE WHEN f.result = 'KO/TKO' OR f.result = 'Submission' THEN 1 END) / NULLIF(COUNT(f.fight_id), 0)) * 100 AS finish_rate
    FROM 
        fight_data f
    JOIN 
        event_data e ON f.event_id = e.event_id
    GROUP BY 
        YEAR(e.event_date)
    ORDER BY 
        year
""")

finish_rate_by_year_pd = finish_rate_by_year.toPandas()


In [ ]:
sns.set(style="whitegrid")
plt.figure(figsize=(12, 6))
sns.barplot(x='year', y='finish_rate', data=finish_rate_by_year_pd, palette='viridis')


plt.title('Finish Rates by Year', fontsize=16)
plt.xlabel('Year', fontsize=14)
plt.ylabel('Finish Rate (%)', fontsize=14)


plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

<p><b>We can see that the odds of a fight ending by submission or knockout has reduced in recent years, making it a less reliable bet that an upcoming fight will end by finish.</b></p>

<h1>Submission Accuracy Analysis:</h1>

In [ ]:

def calc_subs(fighter_name):
    
    if fighter_name == "":
    
        subs = spark.sql("""
            SELECT 
                fd.fighter_f_name, 
                fd.fighter_l_name, 
                SUM(fs.submission_att) AS total_sub_attempts,
                SUM(CASE 
                    WHEN f.winner = fs.fighter_id AND f.result = 'Submission' THEN 1 ELSE 0 END) AS successful_subs,
                (SUM(CASE 
                    WHEN f.winner = fs.fighter_id AND f.result = 'Submission' THEN 1 ELSE 0 END) * 100.0) / NULLIF(SUM(fs.submission_att), 0) AS submission_accuracy
            FROM fight_stats fs
            JOIN fight_data f ON fs.fight_id = f.fight_id
            LEFT JOIN fighter_data fd ON fs.fighter_id = fd.fighter_id
            GROUP BY fd.fighter_f_name, fd.fighter_l_name
            HAVING successful_subs > 0 AND total_sub_attempts > 0
            ORDER BY successful_subs DESC
            LIMIT 10
            """)
    else:
        try:
                f_name, l_name = fighter_name.split()
                subs = spark.sql(f"""
                    SELECT 
                        fd.fighter_f_name, 
                        fd.fighter_l_name, 
                        SUM(fs.submission_att) AS total_sub_attempts,
                        SUM(CASE 
                            WHEN f.winner = fs.fighter_id AND f.result = 'Submission' THEN 1 ELSE 0 END) AS successful_subs,
                        (SUM(CASE 
                            WHEN f.winner = fs.fighter_id AND f.result = 'Submission' THEN 1 ELSE 0 END) * 100.0) / NULLIF(SUM(fs.submission_att), 0) AS submission_accuracy
                    FROM fight_stats fs
                    JOIN fight_data f ON fs.fight_id = f.fight_id
                    LEFT JOIN fighter_data fd ON fs.fighter_id = fd.fighter_id
                    WHERE fd.fighter_f_name = '{f_name}' AND fd.fighter_l_name = '{l_name}'
                    GROUP BY fd.fighter_f_name, fd.fighter_l_name
                    HAVING successful_subs > 0 AND total_sub_attempts > 0
                    ORDER BY successful_subs DESC
                    LIMIT 10
                """)
        except ValueError:
                return "Error: Please provide both first and last name for the fighter."

    return subs

subs = calc_subs("").toPandas()

table = PrettyTable()

# Add columns
table.field_names = ["First Name", "Last Name", "Submission Attempts", "Successful Submissions", "Submission Accuracy"]

# Add rows
for row in subs.itertuples(index=False):
    table.add_row(row)
    
print(table)

In [ ]:
subs = calc_subs("Dustin Poirier").toPandas()
table = PrettyTable()

# Add columns
table.field_names = ["First Name", "Last Name", "Submission Attempts", "Successful Submissions", "Submission Accuracy"]

# Add rows
for row in subs.itertuples(index=False):
    table.add_row(row)
    
print(table)

<p><b>For fighter Dustin Poirier, we can see that his total submissions and submission accuracy is relatively low making it a less reliable bet that he will score a submission win.</b></p>

<h1>Knockdown Rate Analysis:</h1>

In [ ]:
def calc_knockdowns(fighter_name):

    if fighter_name == "":
        knockdown_rate = spark.sql("""
            SELECT 
                fd.fighter_f_name, 
                fd.fighter_l_name,
                SUM(fs.knockdowns) AS total_knockdowns,
                COUNT(fs.fight_id) AS total_fights,
                (SUM(fs.knockdowns) * 1.0 / COUNT(fs.fight_id)) AS knockdown_rate
            FROM fight_stats fs
            JOIN fighter_data fd ON fs.fighter_id = fd.fighter_id
            GROUP BY fd.fighter_f_name, fd.fighter_l_name
            HAVING total_fights > 5
            ORDER BY knockdown_rate DESC
            LIMIT 10
            """)
        
    else:
        try:
            f_name, l_name = fighter_name.split()
            knockdown_rate = spark.sql(f"""
            SELECT 
                fd.fighter_f_name, 
                fd.fighter_l_name,
                SUM(fs.knockdowns) AS total_knockdowns,
                COUNT(fs.fight_id) AS total_fights,
                (SUM(fs.knockdowns) * 1.0 / COUNT(fs.fight_id)) AS knockdown_rate
            FROM fight_stats fs
            JOIN fighter_data fd ON fs.fighter_id = fd.fighter_id
            WHERE fd.fighter_f_name = '{f_name}' AND fd.fighter_l_name = '{l_name}'
            GROUP BY fd.fighter_f_name, fd.fighter_l_name
            HAVING total_fights > 5
            ORDER BY knockdown_rate DESC
            LIMIT 10
            """)
        except ValueError:
            return "Error: Please provide both first and last name for the fighter."

    return knockdown_rate

knockdowns = calc_knockdowns("").toPandas()

table = PrettyTable()

table.field_names = ["First Name", "Last Name", "Total Knockdowns", "Total Fights", "Knockdown Rate"]

for row in knockdowns.itertuples(index=False):
    table.add_row(row)
    
print(table)

In [ ]:
knockdowns = calc_knockdowns("Dustin Poirier").toPandas()

table = PrettyTable()

table.field_names = ["First Name", "Last Name", "Total Knockdowns", "Total Fights", "Knockdown Rate"]

for row in knockdowns.itertuples(index=False):
    table.add_row(row)
    
print(table)

<p><b>For fighter Dustin Poirier, we can see that he has a reliatvely high knockdown rate across his fights making it a reasonable bet that he will score a knockdown in his upcoming fight.</b></p>

<h1>Age/Success Win Rate Analysis:</h1>

In [ ]:


win_rate_by_age_query = """
SELECT 
    YEAR(e.event_date) - YEAR(fd.fighter_dob) AS age_at_fight,
    SUM(CASE WHEN f.winner = fs.fighter_id THEN 1 ELSE 0 END) AS wins,
    COUNT(f.fight_id) AS total_fights,
    (SUM(CASE WHEN f.winner = fs.fighter_id THEN 1 ELSE 0 END) * 100.0) / COUNT(f.fight_id) AS win_rate
FROM fight_stats fs
JOIN fight_data f ON fs.fight_id = f.fight_id
JOIN fighter_data fd ON fs.fighter_id = fd.fighter_id
JOIN event_data e ON f.event_id = e.event_id
WHERE fd.fighter_dob IS NOT NULL AND e.event_date IS NOT NULL
GROUP BY age_at_fight
ORDER BY age_at_fight
"""
win_rate_by_age = spark.sql(win_rate_by_age_query)

In [ ]:
win_rate_by_age_df = win_rate_by_age.toPandas()

# Plot the data
plt.figure(figsize=(10,6))
plt.plot(win_rate_by_age_df['age_at_fight'], win_rate_by_age_df['win_rate'], marker='o', color='b')
plt.title('Win Rate by Fighter\'s Age at Fight')
plt.xlabel('Age at Fight')
plt.ylabel('Win Rate (%)')
plt.grid(True)
plt.show()


<p><b>We can see that win rate typically goes down as fighers age, with a very eratic success rates at higher ages.</b></p>

In [ ]:
win_rate_by_age_query = """
SELECT 
    YEAR(e.event_date) - YEAR(fd.fighter_dob) AS age_at_fight,
    SUM(CASE WHEN f.winner = fs.fighter_id THEN 1 ELSE 0 END) AS wins,
    COUNT(f.fight_id) AS total_fights,
    (SUM(CASE WHEN f.winner = fs.fighter_id THEN 1 ELSE 0 END) * 100.0) / COUNT(f.fight_id) AS win_rate
FROM fight_stats fs
JOIN fight_data f ON fs.fight_id = f.fight_id
JOIN fighter_data fd ON fs.fighter_id = fd.fighter_id
JOIN event_data e ON f.event_id = e.event_id
WHERE fd.fighter_dob IS NOT NULL 
  AND e.event_date IS NOT NULL
  AND fd.fighter_f_name = "Dustin"
  AND fd.fighter_l_name = "Poirier"
GROUP BY age_at_fight
ORDER BY age_at_fight
"""
win_rate_by_age = spark.sql(win_rate_by_age_query)

In [ ]:
win_rate_by_age_pd = win_rate_by_age.toPandas()


plt.figure(figsize=(10, 8))
plt.plot(win_rate_by_age_pd['age_at_fight'], win_rate_by_age_pd['win_rate'], marker='o', linestyle='-')

plt.title('Win Rate by Age for Dustin Poirier')
plt.xlabel('Age at Fight')
plt.ylabel('Win Rate (%)')
plt.xticks(win_rate_by_age_pd['age_at_fight'])
plt.grid()
plt.ylim(-1, 120)
plt.legend()
plt.show()

<p><b>We can see that for Dustin Poirier, his win rate has been quite high in his younger years, but encountered lower success rates more in recent years making it an unreliable bet that he will win his next fight.</b></p>